In [2]:
import torch
from torchvision import datasets, transforms

In [6]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.5], [0.5])])

trainset = datasets.FashionMNIST('./FashionMNISTdata2', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.FashionMNIST('./FashionMNISTdata2', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [7]:
from torch import nn, optim
import torch.nn.functional as F

In [10]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

### Untrained Network

In [11]:
model = Classifier()

images, labels = next(iter(testloader))
ps = torch.exp(model(images))

print(ps.shape)

torch.Size([64, 10])


In [17]:
# Check which classes have the highest probability
top_p, top_class = ps.topk(1, dim=1)

print(top_class[:10, :])
print(top_class.shape)

tensor([[5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5],
        [5]])
torch.Size([64, 1])


In [18]:
labels.shape

torch.Size([64])

In [24]:
equals = top_class == labels.view(*top_class.shape)
equals.shape

torch.Size([64, 1])

In [22]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')

Accuracy: 14.0625%


### Validation Pass

In [33]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30
steps = 0

In [35]:
train_losses, test_losses = [], []

for e in range(epochs):
    running_loss = 0
    
    for images, labels in trainloader:
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Validation Pass goes here
        with torch.no_grad():
            for test_images, test_labels in testloader:
                test_log_ps = model(test_images)
                test_loss += criterion(test_log_ps, test_labels)
                
                test_ps = torch.exp(test_log_ps)
                top_test_p, top_test_class = test_ps.topk(1, dim=1)
                
                test_label_comparison = top_test_class == test_labels.view(*top_test_class.shape)
                accuracy += torch.mean(test_label_comparison.type(torch.FloatTensor))
                
        train_losses.append(running_loss / len(trainloader))
        test_losses.append(test_loss / len(testloader))
        
        print('Epoch: {}/{}..'.format(e+1, epochs),
              'Training Loss: {:.3f}..'.format(running_loss/len(trainloader)),
              'Test Loss: {:.3f}..'.format(test_loss/len(testloader)),
              'Test Accuracy: {:.3f}..'.format(accuracy/len(testloader))
             )

Epoch: 1/30.. Training Loss: 0.393.. Test Loss: 0.465.. Test Accuracy: 0.824..
Epoch: 2/30.. Training Loss: 0.353.. Test Loss: 0.382.. Test Accuracy: 0.866..
Epoch: 3/30.. Training Loss: 0.337.. Test Loss: 0.395.. Test Accuracy: 0.860..
Epoch: 4/30.. Training Loss: 0.318.. Test Loss: 0.389.. Test Accuracy: 0.863..
Epoch: 5/30.. Training Loss: 0.301.. Test Loss: 0.384.. Test Accuracy: 0.867..
Epoch: 6/30.. Training Loss: 0.296.. Test Loss: 0.357.. Test Accuracy: 0.874..
Epoch: 7/30.. Training Loss: 0.283.. Test Loss: 0.381.. Test Accuracy: 0.868..
Epoch: 8/30.. Training Loss: 0.277.. Test Loss: 0.347.. Test Accuracy: 0.879..
Epoch: 9/30.. Training Loss: 0.268.. Test Loss: 0.373.. Test Accuracy: 0.873..
Epoch: 10/30.. Training Loss: 0.264.. Test Loss: 0.375.. Test Accuracy: 0.872..
Epoch: 11/30.. Training Loss: 0.259.. Test Loss: 0.387.. Test Accuracy: 0.875..
Epoch: 12/30.. Training Loss: 0.248.. Test Loss: 0.381.. Test Accuracy: 0.874..
Epoch: 13/30.. Training Loss: 0.247.. Test Loss: 